# 1) Titolo e obiettivi
Lezione 25: PCA e clustering insieme per ridurre la dimensionalita' e rendere piu' robusti i raggruppamenti.
- Obiettivi: applicare PCA prima del clustering, scegliere il numero di componenti, valutare l'impatto su qualita' e velocita', leggere i loadings per interpretare i cluster.
- Cosa useremo: dataset Digits, Wine, Iris e dati sintetici; StandardScaler, PCA, KMeans, DBSCAN, AgglomerativeClustering.
- Prerequisiti: basi di scalatura, PCA, metriche di clustering (silhouette, ARI), conoscenza di KMeans e DBSCAN.


# 2) Teoria concettuale
## 2.1 Perche' combinare PCA e clustering
- PCA comprime l'informazione, riduce il rumore e rende le distanze piu' significative in spazi ad alta dimensionalita'.
- Clustering su poche componenti e' piu' veloce e meno sensibile alla curse of dimensionality.
- Per KMeans: PCA rende le varianze piu' omogenee e facilita centroidi stabili; per DBSCAN riduce la sparizione di densita' utile.
- Se i cluster sono anisotropi, PCA li rende piu' sferici e quindi piu' facili da separare con algoritmi basati su distanza euclidea.


## 2.2 Quante componenti usare
- Criterio di varianza cumulata (es. 0.80, 0.90, 0.95) come compromesso tra perdita di informazione e rumore.
- Scree plot: osservare il gomito dove i benefici marginali calano.
- Verifica empirica: confrontare silhouette/ARI e i tempi di fit tra soglie diverse.
- Loadings: ogni componente e' una combinazione lineare di feature originali; servono a interpretare cosa differenzia i cluster.
- Se servono visualizzazioni, conservare almeno le prime 2-3 componenti anche se la soglia e' piu' alta.


## 2.3 Quando non usare PCA prima del clustering
- Feature gia' semantiche e poche (<15) con scale coerenti: PCA potrebbe offuscare l'interpretabilita'.
- Cluster separati da feature rare/importanti: PCA puo' diluire la separazione eliminando varianza specifica.
- Se DBSCAN lavora gia' bene nello spazio originale (densita' evidente), testare prima senza PCA.
- Regola pratica: confrontare sempre baseline senza PCA e versioni con PCA per verificare che la qualita' non peggiori.


# 3) Schema mentale / mappa decisionale
Workflow di riferimento: load -> clean -> scale -> (opzionale) PCA -> cluster -> valuta -> interpreta.
Decision map testuale:
1. Scala i dati se le feature non hanno la stessa scala.
2. Esegui un clustering senza PCA per avere una baseline di silhouette/ARI e tempi.
3. Prova PCA con soglie 0.80/0.90/0.95 di varianza, confronta metriche e tempi.
4. Se usi DBSCAN, valuta anche combinazioni di eps/min_samples nello spazio PCA.
5. Interpreta i cluster guardando le componenti principali e i loadings piu' forti.
Micro-checklist: niente NaN, forme coerenti dopo PCA, numero cluster atteso, presenza di rumore (-1) se DBSCAN.


# 4) Sezione dimostrativa
Panoramica delle demo:
- Demo 1: Digits con PCA al 90% di varianza + KMeans, metriche e grafici.
- Demo 2: Confronto con/senza PCA su Digits (tempi e qualita').
- Demo 3: Pipeline compatta PCA + KMeans su Iris.
- Demo 4: PCA + DBSCAN per individuare outlier.
- Demo 5: Visualizzazione 3D con PCA su Wine.


## Demo 1 - PCA 90% + KMeans su Digits
Perche': mostrare che ridurre le 64 feature di Digits a poche componenti mantiene la struttura e migliora stabilita'.
Metodi usati (input -> output):
- `StandardScaler`: input array (n_samples x n_features), output array scalato; errore tipico: valori NaN o colonne costanti.
- `PCA(n_components=0.90)`: riduce alle componenti che spiegano il 90% della varianza; errore tipico: n_components troppo alto rispetto ai campioni.
- `KMeans(n_clusters=10)`: richiede dati numerici e n_clusters noto; errore tipico: dati non scalati o cluster non sferici.
Checkpoint attesi: nessun NaN, forma X (1797, 64), forma X_pca (1797, ~20 componenti), varianza cumulata >= 0.90, silhouette > 0.


### Checkpoint visivo per Demo 1
- Scree plot decrescente: il gomito dovrebbe apparire dopo ~15-25 componenti.
- Scatter PC1 vs PC2: punti a gruppi con confini ragionevolmente separati; se tutto sovrapposto, controllare scaler o n_clusters.


In [ ]:
# Demo 1: PCA (90% varianza) + KMeans su Digits
# Intenzione: ridurre dimensionalita', verificare varianza mantenuta, eseguire clustering stabile e valutare silhouette/ARI.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, silhouette_score

np.random.seed(42)
plt.close('all')

# Caricamento dati
X_digits, y_digits = load_digits(return_X_y=True)
print(f"Forma originale X: {X_digits.shape}, y: {y_digits.shape}")
assert X_digits.shape[0] == y_digits.shape[0], "Numero di campioni incoerente"
assert not np.isnan(X_digits).any(), "Sono presenti NaN nel dataset"

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_digits)
print(f"Forma dopo scaling: {X_scaled.shape}")

# PCA al 90% di varianza spiegata
pca = PCA(n_components=0.90, random_state=42)
X_pca = pca.fit_transform(X_scaled)
var_cum = pca.explained_variance_ratio_.sum()
print(f"Componenti tenute: {X_pca.shape[1]}, Varianza cumulata: {var_cum:.3f}")
assert var_cum >= 0.90, "La varianza cumulata e' inferiore alla soglia desiderata"

# KMeans sui dati ridotti
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_pca)
ari = adjusted_rand_score(y_digits, labels)
sil = silhouette_score(X_pca, labels)
print(f"ARI: {ari:.3f}, Silhouette: {sil:.3f}")

# Scree plot e scatter PC1-PC2
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].plot(np.arange(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, marker='o')
axes[0].set_title('Scree plot - varianza per componente')
axes[0].set_xlabel('Componente')
axes[0].set_ylabel('Quota di varianza')
axes[0].axhline(0.05, color='gray', linestyle='--', linewidth=1)

scatter = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='tab10', s=12, alpha=0.7)
axes[1].set_title('KMeans su PC1-PC2')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
plt.colorbar(scatter, ax=axes[1], label='Cluster')
plt.tight_layout()
plt.show()


## Demo 2 - Confronto con e senza PCA
Perche': quantificare il compromesso tra tempo di esecuzione e qualita' del clustering.
Metodi: `Pipeline` con `StandardScaler`, PCA opzionale, `KMeans` fisso a 10 cluster.
Checkpoint: tempo con PCA inferiore o simile, silhouette/ARI non peggiorate; se peggiorano molto, ridurre meno la dimensionalita'.
Errori comuni: dimenticare lo scaler (cluster distorti), PCA con troppe poche componenti (silhouette bassa), silhouette non calcolabile se tutti i punti finiscono nello stesso cluster.


In [ ]:
# Demo 2: Confronto con/senza PCA su Digits
# Intenzione: misurare l'effetto di PCA su tempi e metriche di clustering con KMeans.
import time
from sklearn.pipeline import Pipeline

scenarios = [
    ("Senza PCA", None),
    ("PCA var 0.90", 0.90),
    ("PCA 50 componenti", 50),
]

results = []
for name, n_comp in scenarios:
    steps = [("scaler", StandardScaler())]
    if n_comp is not None:
        steps.append(("pca", PCA(n_components=n_comp, random_state=42)))
    steps.append(("kmeans", KMeans(n_clusters=10, random_state=42, n_init=10)))
    pipe = Pipeline(steps)

    start = time.perf_counter()
    labels = pipe.fit_predict(X_digits)
    elapsed = time.perf_counter() - start

    # Trasformazioni per valutare silhouette nello spazio usato dal modello
    if n_comp is None:
        X_used = pipe.named_steps["scaler"].transform(X_digits)
    else:
        X_scaled_tmp = pipe.named_steps["scaler"].transform(X_digits)
        X_used = pipe.named_steps["pca"].transform(X_scaled_tmp)

    assert X_used.shape[0] == X_digits.shape[0], "Numero di campioni alterato"
    ari = adjusted_rand_score(y_digits, labels)
    sil = silhouette_score(X_used, labels)
    results.append({"scenario": name, "caratteristiche_usate": X_used.shape[1], "silhouette": sil, "ari": ari, "tempo_s": elapsed})

res_df = pd.DataFrame(results)
print(res_df)

best_idx = res_df['silhouette'].idxmax()
print("Miglior silhouette:")
print(res_df.loc[[best_idx]])


## Demo 3 - Pipeline compatta su Iris
Perche': costruire una pipeline riproducibile che includa scaler, PCA e KMeans in un unico oggetto.
Metodi: `Pipeline`, `StandardScaler`, `PCA(n_components=2)`, `KMeans(n_clusters=3)`.
Checkpoint: nessun NaN, forma X_iris (150, 4), forma X_pca (150, 2), silhouette > 0.


In [ ]:
# Demo 3: Pipeline compatta su Iris
# Intenzione: costruire una pipeline scalatore -> PCA -> KMeans e valutarla in 2D.
from sklearn.datasets import load_iris

X_iris, y_iris = load_iris(return_X_y=True)
print(f"Forma X_iris: {X_iris.shape}, y_iris: {y_iris.shape}")
assert not np.isnan(X_iris).any(), "NaN trovati in Iris"
assert X_iris.shape == (150, 4), "Forma inattesa per Iris"

pipe_iris = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2, random_state=42)),
    ("kmeans", KMeans(n_clusters=3, random_state=42, n_init=10)),
])

labels_iris = pipe_iris.fit_predict(X_iris)
X_iris_pca = pipe_iris.named_steps["pca"].transform(pipe_iris.named_steps["scaler"].transform(X_iris))
sil_iris = silhouette_score(X_iris_pca, labels_iris)
print(f"Forma dopo PCA: {X_iris_pca.shape}, Silhouette: {sil_iris:.3f}")
assert X_iris_pca.shape[1] == 2, "La PCA non ha prodotto 2 componenti"
assert sil_iris > 0, "Silhouette non positiva; rivedere n_clusters o n_components"

fig, ax = plt.subplots(figsize=(6, 5))
scatter = ax.scatter(X_iris_pca[:, 0], X_iris_pca[:, 1], c=labels_iris, cmap='viridis', s=30, alpha=0.8)
ax.set_title('Iris: KMeans su PCA 2D')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
plt.colorbar(scatter, ax=ax, label='Cluster')
plt.tight_layout()
plt.show()


## Demo 4 - PCA + DBSCAN per outlier
Perche': usare PCA per compattare i dati e facilitare DBSCAN nell'individuare aree di bassa densita' (outlier).
Metodi: `make_blobs` per dati sintetici, aggiunta di rumore uniforme, `DBSCAN` su spazio PCA.
Checkpoint: esistenza di etichette -1 (rumore), almeno 2 cluster non rumorosi, silhouette calcolata sui soli punti non rumorosi.


In [ ]:
# Demo 4: PCA + DBSCAN per outlier detection
# Intenzione: usare PCA per compattare dati sintetici e permettere a DBSCAN di evidenziare il rumore.
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN

X_blobs, _ = make_blobs(n_samples=500, centers=4, n_features=6, cluster_std=1.2, random_state=42)
noise = np.random.uniform(low=-8, high=8, size=(20, 6))
X_mix = np.vstack([X_blobs, noise])
print(f"Forma dati con rumore: {X_mix.shape}")
assert not np.isnan(X_mix).any(), "NaN nei dati sintetici"

pipe_dbscan = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2, random_state=42)),
])
X_mix_pca = pipe_dbscan.fit_transform(X_mix)
print(f"Forma dopo PCA: {X_mix_pca.shape}")
assert X_mix_pca.shape[0] == X_mix.shape[0], "Numero di campioni alterato dalla pipeline"

labels_dbscan = DBSCAN(eps=0.9, min_samples=5).fit_predict(X_mix_pca)
unique_labels = np.unique(labels_dbscan)
print(f"Etichette trovate: {unique_labels}")

mask = labels_dbscan != -1
if mask.sum() > 0 and np.unique(labels_dbscan[mask]).size > 1:
    sil_db = silhouette_score(X_mix_pca[mask], labels_dbscan[mask])
else:
    sil_db = np.nan
print(f"Silhouette (solo cluster non rumore): {sil_db}")

fig, ax = plt.subplots(figsize=(7, 6))
scatter = ax.scatter(X_mix_pca[:, 0], X_mix_pca[:, 1], c=labels_dbscan, cmap='tab10', s=20, alpha=0.8)
ax.set_title('DBSCAN su PCA 2D (rumore = -1)')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
plt.colorbar(scatter, ax=ax, label='Etichetta DBSCAN')
plt.tight_layout()
plt.show()


## Demo 5 - PCA 3D su Wine
Perche': visualizzare i cluster su tre componenti e valutare quanta varianza manteniamo.
Metodi: `PCA(n_components=3)` e `KMeans(n_clusters=3)`.
Checkpoint: varianza cumulata > 0.60, silhouette > 0, grafico 3D con cluster separabili.


In [ ]:
# Demo 5: PCA 3D + KMeans su Wine
# Intenzione: visualizzare cluster Wine su 3 componenti principali e valutarne la separazione.
from mpl_toolkits.mplot3d import Axes3D  # necessaria per il projection='3d'
from sklearn.datasets import load_wine

X_wine, y_wine = load_wine(return_X_y=True)
print(f"Forma X_wine: {X_wine.shape}, y_wine: {y_wine.shape}")
assert not np.isnan(X_wine).any(), "NaN trovati nel dataset Wine"

pipe_wine = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=3, random_state=42)),
    ("kmeans", KMeans(n_clusters=3, random_state=42, n_init=10)),
])
labels_wine = pipe_wine.fit_predict(X_wine)
X_wine_pca = pipe_wine.named_steps["pca"].transform(pipe_wine.named_steps["scaler"].transform(X_wine))
var_wine = pipe_wine.named_steps["pca"].explained_variance_ratio_.sum()
sil_wine = silhouette_score(X_wine_pca, labels_wine)
print(f"Forma dopo PCA: {X_wine_pca.shape}, Varianza cumulata: {var_wine:.3f}, Silhouette: {sil_wine:.3f}")
assert var_wine >= 0.60, "Varianza cumulata troppo bassa: aumenta n_components"

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_wine_pca[:, 0], X_wine_pca[:, 1], X_wine_pca[:, 2], c=labels_wine, cmap='tab10', s=30, alpha=0.8)
ax.set_title('Wine: KMeans su PCA 3D')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
fig.colorbar(scatter, ax=ax, label='Cluster')
plt.tight_layout()
plt.show()


# 5) Esercizi svolti (passo-passo)
## Esercizio 25.1 - Customer segmentation sintetica
Obiettivo: generare un dataset sintetico 1000x20, ridurre al 90% di varianza con PCA e clusterizzare in 5 gruppi con KMeans.
Perche': esercitarsi con un flusso end-to-end includendo controlli su forma, NaN e metriche.


In [ ]:
# Esercizio 25.1: Customer segmentation sintetica
# Step-by-step: genera dati, scala, PCA al 90%, KMeans a 5 cluster, valuta silhouette/ARI.
from sklearn.datasets import make_blobs

X_synth, y_synth_true = make_blobs(n_samples=1000, centers=5, n_features=20, cluster_std=2.0, random_state=42)
print(f"Forma dati sintetici: {X_synth.shape}")
assert not np.isnan(X_synth).any(), "NaN nei dati sintetici"

pipe_synth = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.90, random_state=42)),
    ("kmeans", KMeans(n_clusters=5, random_state=42, n_init=10)),
])

labels_synth = pipe_synth.fit_predict(X_synth)
X_synth_pca = pipe_synth.named_steps["pca"].transform(pipe_synth.named_steps["scaler"].transform(X_synth))
var_synth = pipe_synth.named_steps["pca"].explained_variance_ratio_.sum()
sil_synth = silhouette_score(X_synth_pca, labels_synth)
ari_synth = adjusted_rand_score(y_synth_true, labels_synth)
print(f"Componenti tenute: {X_synth_pca.shape[1]}, Varianza: {var_synth:.3f}, Silhouette: {sil_synth:.3f}, ARI: {ari_synth:.3f}")

# Sanity check: tutti i cluster presenti
unique_labels = np.unique(labels_synth)
print(f"Cluster trovati: {unique_labels}")
assert len(unique_labels) == 5, "Non sono stati trovati 5 cluster"


## Esercizio 25.2 - Confronto algoritmi su Digits
Obiettivo: confrontare KMeans, AgglomerativeClustering e DBSCAN sullo spazio PCA (80% di varianza) del dataset Digits.
Perche': mostrare come PCA impatta algoritmi diversi e come leggere ARI/silhouette, gestendo il rumore di DBSCAN.


In [ ]:
# Esercizio 25.2: Confronto algoritmi su Digits (PCA 80%)
# Intenzione: confrontare KMeans, Agglomerative e DBSCAN dopo PCA, valutando silhouette e ARI.
from sklearn.cluster import AgglomerativeClustering

scaler_80 = StandardScaler()
pca_80 = PCA(n_components=0.80, random_state=42)
X_digits_pca80 = pca_80.fit_transform(scaler_80.fit_transform(X_digits))
print(f"Forma PCA 80%: {X_digits_pca80.shape}, Varianza: {pca_80.explained_variance_ratio_.sum():.3f}")

models = [
    ("KMeans", KMeans(n_clusters=10, random_state=42, n_init=10)),
    ("Agglomerative", AgglomerativeClustering(n_clusters=10)),
    ("DBSCAN", DBSCAN(eps=3.0, min_samples=5)),
]

rows = []
for name, model in models:
    labels = model.fit_predict(X_digits_pca80)
    mask = labels != -1
    if mask.sum() > 0 and np.unique(labels[mask]).size > 1:
        sil = silhouette_score(X_digits_pca80[mask], labels[mask])
    else:
        sil = np.nan
    ari = adjusted_rand_score(y_digits, labels)
    rows.append({"modello": name, "silhouette": sil, "ari": ari, "etichette_trovate": np.unique(labels).size})

res_models = pd.DataFrame(rows)
print(res_models)


## Esercizio 25.3 - Mini tuning PCA + DBSCAN
Obiettivo: cercare combinazioni di (n_components, eps, min_samples) che migliorano silhouette su Digits ridotto con PCA.
Perche': illustrare un micro-grid manuale senza nuove librerie, con controlli di forma e cluster presenti.


In [ ]:
# Esercizio 25.3: Mini tuning PCA + DBSCAN su Digits
# Intenzione: esplorare manualmente combinazioni di n_components, eps e min_samples per migliorare silhouette.
param_grid = {
    "n_components": [0.70, 0.80, 0.90],
    "eps": [2.5, 3.0, 3.5],
    "min_samples": [3, 5],
}

best = None
candidates = []
for n_comp in param_grid["n_components"]:
    pca_tune = PCA(n_components=n_comp, random_state=42)
    X_tune = pca_tune.fit_transform(scaler.fit_transform(X_digits))
    for eps in param_grid["eps"]:
        for ms in param_grid["min_samples"]:
            labels = DBSCAN(eps=eps, min_samples=ms).fit_predict(X_tune)
            mask = labels != -1
            if mask.sum() < 2 or np.unique(labels[mask]).size < 2:
                continue
            sil = silhouette_score(X_tune[mask], labels[mask])
            ari = adjusted_rand_score(y_digits, labels)
            row = {"n_components": n_comp, "eps": eps, "min_samples": ms, "silhouette": sil, "ari": ari, "etichette_trovate": np.unique(labels).size}
            candidates.append(row)
            if best is None or sil > best["silhouette"]:
                best = row

if candidates:
    df_tuning = pd.DataFrame(candidates).sort_values(by="silhouette", ascending=False)
    print("Top 5 combinazioni per silhouette:")
    print(df_tuning.head())
    print("Migliore combinazione:")
    print(best)
else:
    print("Nessuna combinazione ha prodotto almeno due cluster non rumorosi.")


# 6) Conclusione operativa
Risultati operativi:
- PCA prima del clustering riduce tempi e spesso migliora silhouette/ARI mantenendo la struttura (Digits e Wine).
- Confrontare sempre baseline senza PCA con soglie diverse di varianza per scegliere n_components.
- DBSCAN beneficia di PCA per rumore e densita', ma richiede tuning di eps/min_samples dopo la riduzione.

Metodi spiegati (cosa fa, input/output, quando usarlo):
- `StandardScaler`: centra e scala colonne numeriche; input matrice n_samples x n_features, output stessa forma; usalo prima di PCA/KMeans.
- `PCA`: riduce dimensionalita' preservando varianza; input dati scalati, output n_samples x n_components; evita se vuoi interpretare feature originali senza mescolarle.
- `KMeans`: clustering per centroidi; input numerico, output etichette; funziona bene con cluster sferici e dati scalati.
- `AgglomerativeClustering`: clustering gerarchico bottom-up; input numerico, output etichette; utile se vuoi dendrogrammi o cluster non sferici.
- `DBSCAN`: clustering basato su densita' con etichetta -1 per rumore; input numerico, output etichette; funziona bene con cluster di forma arbitraria, sensibile a eps/min_samples.
- `silhouette_score`: misura coesione/separazione dei cluster (range -1..1); richiede almeno 2 cluster; non usare se tutti i punti stanno in un cluster unico.
- `adjusted_rand_score`: confronta etichette previste con etichette vere; input due vettori di lunghezza n_samples; output da -1 a 1.

Errori comuni e debug rapido:
- Silhouette negativa o molto bassa -> probabile n_components troppo basso o n_clusters sbagliato; prova piu' componenti o rivedi il numero di cluster.
- Tutti i punti in un solo cluster con DBSCAN -> eps troppo piccolo o min_samples troppo alto; allarga eps o riduci min_samples.
- Varianza cumulata sotto la soglia richiesta -> controlla scaler o aumenta leggermente n_components.
- ARI molto basso rispetto alla baseline senza PCA -> prova una soglia di varianza piu' alta o evita PCA se le feature sono gia' informative.


# 7) Checklist di fine lezione
- [ ] Ho scalato i dati prima di PCA e clustering.
- [ ] Ho scelto `n_components` confrontando varianza e metriche (silhouette/ARI).
- [ ] Ho verificato forme coerenti dopo PCA e che non ci siano NaN.
- [ ] Ho controllato tempi di fit con e senza PCA.
- [ ] Ho interpretato i cluster guardando le componenti e i loadings piu' pesanti.
- [ ] Ho gestito il rumore di DBSCAN verificando la presenza di etichette -1.

Glossario (termini usati nel notebook):
- PCA: tecnica di riduzione dimensionale basata su varianza massima.
- Principal component (PC): nuova variabile lineare che massimizza la varianza.
- Varianza spiegata: quota di varianza coperta dalle componenti scelte.
- Scree plot: grafico della varianza spiegata per componente.
- Loadings: pesi delle feature originali su ogni componente.
- n_components: numero di componenti mantenute o soglia di varianza.
- Silhouette score: metrica di coesione/separazione dei cluster.
- Adjusted Rand Index (ARI): accordo tra etichette previste e vere.
- KMeans: clustering per centroidi con n_clusters fissato.
- DBSCAN: clustering per densita' con etichetta -1 per il rumore.
- AgglomerativeClustering: clustering gerarchico bottom-up.
- Scaling: trasformazione per avere media 0 e deviazione standard 1 per ogni feature.
- Punti rumore: punti etichettati -1 da DBSCAN.
- eps: raggio di vicinanza usato da DBSCAN.
- min_samples: numero minimo di vicini per formare un core point in DBSCAN.
- Curse of dimensionality: fenomeno di distanze meno informative in spazi ad alta dimensione.


# 8) Changelog didattico
- Ristrutturata la lezione nelle 8 sezioni richieste con titoli in italiano.
- Aggiunte spiegazioni operative prima di ogni blocco di codice e checklist di checkpoint attesi.
- Inseriti controlli di forma, NaN, varianza cumulata, silhouette e ARI con assert dove rilevante.
- Approfondite le demo con confronti PCA vs no PCA e visualizzazioni PCA 2D/3D.
- Resi gli esercizi guidati passo-passo con note su errori comuni e interpretazione dei risultati.
- Aggiunti metodi spiegati, errori comuni, checklist e glossario interno alla lezione.
